In [1]:
from MertonPDEModel import MertonPDEModel, BalanceSheetGuesser
import numpy as np
import pandas as pd

/Users/patrickzhao/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
data = pd.read_csv('data.csv')
data.head()

,Ticker,Equity_Q2,Equity_Q3,Cap_Q3,Cap_Q2,Strike,Maturity,IV,Debt_Q2,Debt_Q3,Asset_Q3
0,GE,0.019345,0.019021,317.31,272.95,260,1,0.3294,0.018887,0.2,0.13
1,GE,0.019345,0.019021,317.31,272.95,270,1,0.3280,0.018887,0.2,0.13
2,GE,0.019345,0.019021,317.31,272.95,280,1,0.3195,0.018887,0.2,0.13
3,GE,0.019345,0.019021,317.31,272.95,290,1,0.3183,0.018887,0.2,0.13
4,GE,0.019345,0.019021,317.31,272.95,300,1,0.3013,0.018887,0.2,0.13


In [3]:
GE = data[data['Ticker'] == 'GE']
AAPL = data[data['Ticker'] == 'AAPL']
JPM = data[data['Ticker'] == 'JPM']

In [4]:
def run(data):
    print(f"--- Processing {data.iloc[0]['Ticker']} ---")
    for i in range(len(data)):
        print(f"--- Processing option with strike {data.iloc[i]['Strike']} ---")
        row = data.iloc[i]
        obs_market_cap = row['Cap_Q2']   # $85 Billion
        obs_iv = row['IV']           # 32% Volatility
        r_rate = 0.04           # 4.0% Risk free
        T_maturity = row['Maturity']        # Avg debt maturity
        debt_last_Q = row['Debt_Q2']      # Debt known from last quarter

        print(f"--- 📅 Calibration Step ---")
        guesser = BalanceSheetGuesser(r_rate, T_maturity, debt_last_Q)
        prediction = guesser.guess(obs_market_cap, obs_iv)

        print(f"Observed Equity: ${obs_market_cap}B, Vol: {obs_iv:.1%}")
        print(f"Implied Firm Assets (V): ${prediction['Implied_V']:.2f} B")
        print(f"Implied Asset Volatility: {prediction['Implied_Asset_Vol']:.2%}")

        # Validation
        actual_debt_book = row['Debt_Q3'] # Secretly borrowed 10B more
        print(f"\n--- 🔍 Reality Check ---")
        print(f"The model 'thought' debt was ${debt_last_Q}B (Face).")
        print(f"The actual debt is ${actual_debt_book}B.")
        print(f"Implied Market Value of Debt: ${prediction['Implied_Debt_Value_Market']:.2f}B")

        # If Implied Debt Value < Actual Face Value, the market is pricing in distress
        if prediction['Implied_Debt_Value_Market'] < actual_debt_book * 0.95:
                print("Result: The market is pricing this debt at a discount (Distress Risk).")
        else:
                print("Result: The market views this debt as safe.")

In [5]:
run(GE)

--- Processing GE ---
--- Processing option with strike 260 ---
--- 📅 Calibration Step ---
Observed Equity: $272.95B, Vol: 32.9%
Implied Firm Assets (V): $272.97 B
Implied Asset Volatility: 32.94%

--- 🔍 Reality Check ---
The model 'thought' debt was $0.018887B (Face).
The actual debt is $0.2B.
Implied Market Value of Debt: $0.02B
Result: The market is pricing this debt at a discount (Distress Risk).
--- Processing option with strike 270 ---
--- 📅 Calibration Step ---
Observed Equity: $272.95B, Vol: 32.8%
Implied Firm Assets (V): $272.97 B
Implied Asset Volatility: 32.80%

--- 🔍 Reality Check ---
The model 'thought' debt was $0.018887B (Face).
The actual debt is $0.2B.
Implied Market Value of Debt: $0.02B
Result: The market is pricing this debt at a discount (Distress Risk).
--- Processing option with strike 280 ---
--- 📅 Calibration Step ---
Observed Equity: $272.95B, Vol: 31.9%
Implied Firm Assets (V): $272.97 B
Implied Asset Volatility: 31.95%

--- 🔍 Reality Check ---
The model 'tho

In [6]:
run(AAPL)

--- Processing AAPL ---
--- Processing option with strike 270 ---
--- 📅 Calibration Step ---
Observed Equity: $3050.0B, Vol: 26.1%
Implied Firm Assets (V): $3050.10 B
Implied Asset Volatility: 26.08%

--- 🔍 Reality Check ---
The model 'thought' debt was $0.101698B (Face).
The actual debt is $0.99B.
Implied Market Value of Debt: $0.10B
Result: The market is pricing this debt at a discount (Distress Risk).
--- Processing option with strike 275 ---
--- 📅 Calibration Step ---
Observed Equity: $3050.0B, Vol: 25.7%
Implied Firm Assets (V): $3050.10 B
Implied Asset Volatility: 25.69%

--- 🔍 Reality Check ---
The model 'thought' debt was $0.101698B (Face).
The actual debt is $0.99B.
Implied Market Value of Debt: $0.10B
Result: The market is pricing this debt at a discount (Distress Risk).
--- Processing option with strike 280 ---
--- 📅 Calibration Step ---
Observed Equity: $3050.0B, Vol: 25.3%
Implied Firm Assets (V): $3050.10 B
Implied Asset Volatility: 25.28%

--- 🔍 Reality Check ---
The mod

In [7]:
run(JPM)

--- Processing JPM ---
--- Processing option with strike 305 ---
--- 📅 Calibration Step ---
Observed Equity: $797.18B, Vol: 25.8%
Implied Firm Assets (V): $797.65 B
Implied Asset Volatility: 25.73%

--- 🔍 Reality Check ---
The model 'thought' debt was $0.485095B (Face).
The actual debt is $0.5B.
Implied Market Value of Debt: $0.47B
Result: The market is pricing this debt at a discount (Distress Risk).
--- Processing option with strike 310 ---
--- 📅 Calibration Step ---
Observed Equity: $797.18B, Vol: 24.8%
Implied Firm Assets (V): $797.65 B
Implied Asset Volatility: 24.82%

--- 🔍 Reality Check ---
The model 'thought' debt was $0.485095B (Face).
The actual debt is $0.5B.
Implied Market Value of Debt: $0.47B
Result: The market is pricing this debt at a discount (Distress Risk).
--- Processing option with strike 315 ---
--- 📅 Calibration Step ---
Observed Equity: $797.18B, Vol: 24.2%
Implied Firm Assets (V): $797.65 B
Implied Asset Volatility: 24.19%

--- 🔍 Reality Check ---
The model 'th